- **Nom**: David Morillo Massagué
- **NIU**: 1666540

### 1. LFSR implementation

In [ ]:
import numpy as np

class LFSR():
    def __init__(self, state: np.ndarray, pol: np.ndarray):
        self.state = state
        self.pol = pol
        self.n = len(state)

    def next(self) -> int:
        output= self.state[-1] # The output bit is the last bit of the state
        next_bit = 0
        for i in range(self.n):
            next_bit ^= self.state[i] & self.pol[i]
        # Shift the state to the right and insert the new bit at the beginning
        self.state = np.roll(self.state, 1) # Shift right
        self.state[0] = next_bit
        return output

    def next_n(self, n: int) -> np.ndarray:
        # Generate the next n bits of the LFSR sequence
        bits = np.zeros(n, dtype=int)
        for i in range(n):
            bits[i] = self.next()
        return bits

test_s1 = np.array([1,0,1])
test_c1 = np.array([1,1,1])
test_lfsr1 = LFSR(test_s1, test_c1)
test_seq1 = np.array([test_lfsr1.next() for _ in range(20)])
print(f"Test sequence 1:{test_seq1}")
test_s2 = np.array([1,0,0,0,1])
test_c2 = np.array([1,1,0,0,1])
test_lfsr2 = LFSR(test_s2, test_c2)
test_seq2 = np.array([test_lfsr2.next() for _ in range(20)])
print(f"Test sequence 2:{test_seq2}")


Test sequence 1:[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
Test sequence 2:[1 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 1 0]


### 2. Sequence generation

In [20]:
s1 = np.array([1,1,1,1,1,1])
c1 = np.array([1,0,0,0,0,1])
lfsr_1 = LFSR(s1, c1)
a1000 = lfsr_1.next_n(1000)

s2 = np.array([1,0,0,1,1,1])
c2 = np.array([1,0,0,1,1,0])
lfsr_2 = LFSR(s2, c2)
b1000 = lfsr_2.next_n(1000)

s3 = np.array([1,0,0,0,0,0])
c3 = np.array([0,0,0,0,1,0])
lfsr_3 = LFSR(s3, c3)
c1000 = lfsr_3.next_n(1000)

### 3. NIST test for RNGs

In [45]:
import math
from scipy.special import gammainc
from typing import Tuple

# Test 1: Frequencia (Monobit)
def rng_test_1(seq: np.ndarray) -> Tuple[bool, float]:
    total_bits = len(seq)
    num_ones = sum(seq)
    s_obs = abs(2 * num_ones - total_bits) / math.sqrt(total_bits)

    p_value = math.erfc(s_obs / math.sqrt(2))
    return p_value >= 0.01, p_value

# Test 2: Frequencia blocs
def rng_test_2(seq: np.ndarray, m: int = 10) -> Tuple[bool, float]:
    total_bits = len(seq)
    num_blocks = total_bits // m
    seq = seq[:num_blocks * m]
    blocks = seq.reshape((num_blocks, m))
    proportions = np.mean(blocks, axis=1)
    x_obs = 4 * m * sum((proportions - 0.5) ** 2)

    p_value = float(gammainc(num_blocks / 2, x_obs / 2))
    return p_value >= 0.01, p_value

# Test 3: Runs (rachas)
def rng_test_3(seq: np.ndarray) -> Tuple[bool, float]:
    total_bits = len(seq)
    pi = sum(seq) / total_bits

    if abs(pi - 0.5) > 0.25:
        return False, 0.0

    v_obs = 1
    for i in range(1, total_bits):
        if seq[i] != seq[i - 1]:
            v_obs += 1

    num = abs(v_obs - (2 * total_bits * pi * (1 - pi)))
    den = 2 * math.sqrt(2 * total_bits) * pi * (1 - pi)
    p_value = math.erfc(num / den)

    return p_value >= 0.01, p_value

for i, sequence in enumerate([a1000, b1000, c1000]):
    test_1 = rng_test_1(sequence)
    test_2 = rng_test_2(sequence, m=10)
    test_3 = rng_test_3(sequence)

    print(f"Testing sequence: {i+1}")
    print(f"Frequency Test:             {test_1[0]}, p-value: {test_1[1]}")
    print(f"Frequency within a block:   {test_2[0]}, p-value: {test_2[1]}")
    print(f"Runs:                       {test_3[0]}, p-value: {test_3[1]}")

    if test_1[0] and test_2[0] and test_3[0]:
        print(f"Sequence {i+1} is random.\n")
    else:
        print(f"Sequence {i+1} is not random.\n")

Testing sequence: 1
Frequency Test:             True, p-value: 0.48661604576405054
Frequency within a block:   True, p-value: 0.08452672009769412
Runs:                       True, p-value: 0.5169366858432001
Sequence 1 is random.

Testing sequence: 2
Frequency Test:             True, p-value: 0.9495709711511051
Frequency within a block:   False, p-value: 3.04279258146972e-100
Runs:                       True, p-value: 0.999899074295685
Sequence 2 is not random.

Testing sequence: 3
Frequency Test:             False, p-value: 8.435768358353881e-81
Frequency within a block:   True, p-value: 1.0
Runs:                       False, p-value: 0.0
Sequence 3 is not random.

